In [1]:
# %pip install crewai crewai_tools langchain langchain_community langchain_ollama streamlit duckduckgo-search

In [19]:
from crewai import LLM
from langchain_ollama.llms import OllamaLLM

llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)

In [20]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults 
import json


@tool
def search_web_tool(query: str):
    """
    Searches the web and returns results.
    """
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)


In [21]:
from crewai import Agent

# Agent Resercher
guide_expert= Agent( 
    role="City Local Guide Expert",
    goal="Provides information on things to do in the city based on the user's interests.",
    backstory="""A local expert with a passion for sharing the best experiences and hidden gems of their city.""",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=LLM(model="ollama/llama3.2",base_url="http://localhost:11434"),  #ChatOpenAI(temperature=0, model="gpt-4o-mini"),
    allow_delegation=False,
    )

In [22]:
# Agent city expert
location_expert = Agent(
    role="Travel Trip Expert",
    goal="Gather helpful information about to the city and city during travel.",
    backstory="""A seasoned traveler who has explored various destinations and knows the ins and outs of travel logistics.""",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=LLM(model="ollama/llama3.2",base_url="http://localhost:11434"),
    allow_delegation=False,
    )


In [23]:
planner_expert = Agent(
    role="Travel Planning Expert",
    goal="Compiles all gathered information to provide a comprehensive travel plan.",
    backstory="""
    You are a professional guide with a passion for travel.
    An organizational wizard who can turn a list of possibilities into a seamless itinerary.
    """,
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=LLM(model="ollama/llama3.2",base_url="http://localhost:11434"),
    allow_delegation=False,
    )


In [24]:
from datetime import datetime
from crewai import Task

from_city = "Pakistan"
destination_city = "Paris"
date_from = "10th May 2025"
date_to = "20th May 2025"
interests = "foodie and nature beauty"

location_task = Task(
    description=f"""
    This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
    consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
    
    Traveling from : {from_city}
    Destination city : {destination_city}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1.Respond in ENGLISH Always.
    """,
    expected_output=f"""
    Respond in ENGLISH Always.
    In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily living expenses, and practical travel tips to ensure a smooth journey.
    """,
    agent=location_expert,
    output_file='city_report.md',
)





In [25]:
guide_task = Task(
    description=f"""
    Respond in ENGLISH Always.
    Tailored to the traveler's personal {interests}, this task focuses on creating an engaging and informative guide to the city's attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor activities that align with the user's preferences such {interests}. The guide also highlights seasonal events and festivals that might be of interest during the traveler's visit.
    Destination city : {destination_city}
    interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. Respond in ENGLISH Always.
    """,
    expected_output=f"""
    An interactive markdown report that presents a personalized itinerary of activities and attractions, complete with descriptions, locations, and any necessary reservations or tickets.
    """,

    agent=guide_expert,
    output_file='guide_report.md',
)


In [26]:
planner_task = Task(
    description=f"""
    This task synthesizes all collected information into a detaileds introduction to the city (description of city and presentation, in 3 pragraphes) cohesive and practical travel plan. and takes into account the traveler's schedule, preferences, and budget to draft a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy navigation.
    Destination city : {destination_city}
    interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. Respond in ENGLISH Always.
    """,
    expected_output="""
    Respond in ENGLISH Always.
    A rich markdown document with emojis on each title and subtitle, that :
    In markdown format : 
    # Welcome to {destination_city} :
    A 4 paragraphes markdown formated including :
    - a curated articles of presentation of the city, 
    - a breakdown of daily living expenses, and spots to visit.
    # Here's your Travel Plan to {destination_city} :
    Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's highlights based on the guide's recommendations
    """,
    context=[location_task, guide_task],
    #context=context,
    agent=planner_expert,
        output_file='travel_plan.md',
        )


In [27]:
# Task: Location
def location_task(agent, from_city, destination_city, date_from, date_to):
    return Task(
        description=f"""
        This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
        consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
        
        Traveling from : {from_city}
        Destination city : {destination_city}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. Respond in ENGLISH Always.
        """,
        expected_output=f"""
        Respond in ENGLISH Always.
        In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily living expenses, and practical travel tips to ensure a smooth journey.
        """,
        agent=agent,
        output_file='city_report.md',
    )

# Task: Location
def guide_task(agent, destination_city, interests, date_from, date_to):    
    return Task(
        description=f"""
        Respond in ENGLISH Always
        Tailored to the traveler's personal {interests}, this task focuses on creating an engaging and informative guide to the city's attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor activities that align with the user's preferences such {interests}. The guide also highlights seasonal events and festivals that might be of interest during the traveler's visit.
        Destination city : {destination_city}
        interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. Respond in ENGLISH Always.
        """,
        expected_output=f"""
        An interactive markdown report that presents a personalized itinerary of activities and attractions, complete with descriptions, locations, and any necessary reservations or tickets.
        """,

        agent=agent,
        output_file='guide_report.md',
    )


# Task: Planner
def planner_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=f"""
        This task synthesizes all collected information into a detaileds introduction to the city (description of city and presentation, in 3 pragraphes) cohesive and practical travel plan. and takes into account the traveler's schedule, preferences, and budget to draft a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy navigation.
        Destination city : {destination_city}
        interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. Respond in ENGLISH.
        """,
        expected_output="""
        if the {destination_city} is in a French country : Respond in FRENCH.
        A rich markdown document with emojis on each title and subtitle, that :
        In markdown format : 
        # Welcome to {destination_city} :
        A 4 paragraphes markdown formated including :
        - a curated articles of presentation of the city, 
        - a breakdown of daily living expenses, and spots to visit.
        # Here's your Travel Plan to {destination_city} :
        Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's highlights based on the guide's recommendations
        """,
        #context=[location_task, guide_task],
        context=context,
        agent=agent,
        output_file='travel_plan.md',
        )


In [28]:

location_task = location_task(
  location_expert,
  from_city,
  destination_city,
  date_from,
  date_to
)

guide_task = guide_task(
  guide_expert,
  destination_city,
  interests,
  date_from,
  date_to
)

planner_task = planner_task(
  [location_task, guide_task],
  planner_expert,
  destination_city,
  interests,
  date_from,
  date_to,
)



In [29]:
from crewai import Crew, Process
import litellm
crew = Crew(
agents=[location_expert, guide_expert, planner_expert],
tasks=[location_task, guide_task, planner_task],
process=Process.sequential,
full_output=True,
share_crew=False,
verbose=True
)

litellm._turn_on_debug()
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c9def371-6a81-4fd3-a5e2-a3ec5eec9334                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 897ad4ec-64f6-44ec-b014-bfc007db8d81
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 897ad4ec-64f6-44ec-b014-bfc007db8d81
       Status: Executing Task...
    └── 🤖 Agent: Travel Trip Expert
            Status: In Progress

10:35:10 - LiteLLM:DEBUG: utils.py:332 - 

DEBUG:LiteLLM:

10:35:10 - LiteLLM:DEBUG: utils.py:332 - Request to litellm:
DEBUG:LiteLLM:Request to litellm:
10:35:10 - LiteLLM:DEBUG: utils.py:332 - litellm.completion(model='ollama/llama3.2', messages=[{'role': 'system', 'content': 'You are Travel Trip Expert. A seasoned traveler who has explored various destinations and knows the ins and outs of travel logistics.\nYour personal goal is: Gather helpful information about to the city and city during travel.\nYou ONLY have access to the following tools, and should NEVER make up tools that are not listed here:\n\nTool Name: search_web_tool\nTool Arguments: {\'query\': {\'description\': None, \'type\': \'str\'}}\nTool Description: \n    Searches the web and returns results.\n    \n\nIMPORTANT: Use the following format in your response:\n\n```\nThought: you should always think about what to do\nAction: the action to take, only one name of [search_web_tool], just the name, exactly as it\'s writte

# Agent: Travel Trip Expert
## Task: 
        This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
        consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.

        Traveling from : Pakistan
        Destination city : Paris
        Arrival Date : 10th May 2025
        Departure Date : 20th May 2025

        Follow this rules : 
        1. Respond in ENGLISH Always.
        


...



# Agent: Travel Planning Expert
## Final Answer: 
Action: {'query': 'Paris travel guide for foodie and nature beauty trips'}





🚀 Crew: crew
├── 📋 Task: 897ad4ec-64f6-44ec-b014-bfc007db8d81
│      Assigned to: Travel Trip Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Travel Trip Expert
│           Status: ✅ Completed
├── 📋 Task: 32910702-f2b2-49f5-866b-d66322a91464
│      Assigned to: City Local Guide Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: City Local Guide Expert
│           Status: ✅ Completed
└── 📋 Task: 50989928-a9f8-46cb-a074-199933e65adb
       Status: Executing Task...
    └── 🤖 Agent: Travel Planning Expert
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 897ad4ec-64f6-44ec-b014-bfc007db8d81
│      Assigned to: Travel Trip Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Travel Trip Expert
│           Status: ✅ Completed
├── 📋 Task: 32910702-f2b2-49f5-866b-d66322a91464
│      Assigned to: City Local Guide Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: City Local Guide Expert
│           Status: ✅ Completed
└── 📋 Task: 50989928-a9f8-46cb-a074-199933e65adb
       Assigned to: Travel Planning Expert
       Status: ✅ Completed
    └── 🤖 Agent: Travel Planning Expert
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 50989928-a9f8-46cb-a074-199933e65adb                                                                     │
│  Agent: Travel Planning Expert                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c9def371-6a81-4fd3-a5e2-a3ec5eec9334                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯